# LEMAIYAN HEIGHTS RENT AUTOMATION BOT (PROOF OF CONCEPT, PROTOTYPE, DEPLOYMENT)

<img src= 'images\rent bot.jpg' width='400'>

## PROOF OF CONCEPT

* Random generated dataset to simulate results

In [7]:
import random
from faker import Faker
from pathlib import Path
import string

fake = Faker()

# Define possible account codes (A1-A6, B1-B6, ..., G1-G6)
accounts = [f"{l}{n}" for l in "ABCDEFG" for n in range(1, 7)]

email_template = ("Dear Customer, your payment of KES {amount} for account: PAYLEMAIYAN #{code} "
                  "has been received from {name} {phone} on {date_time}. "
                  "M-Pesa Ref: {mpesa_ref}")
def random_ref_code(length=10):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
dummy_emails = []
for _ in range(200):
    amount = f"{random.randint(5, 20) * 1000}.00"
    code = random.choice(accounts)
    name = fake.name()
    phone = f"{random.randint(700, 799)}****{random.randint(100,999)}"
    date_time = fake.date_time_this_year().strftime('%d/%m/%Y %I:%M %p')
    mpesa_ref = random_ref_code(10)
    email_text = email_template.format(
        amount=amount,
        code=code,
        name=name,
        phone=phone,
        date_time=date_time,
        mpesa_ref=mpesa_ref
    )
    dummy_emails.append(email_text)

# Save to data/dummy_emails_200.txt
out_path = Path("data/dummy_emails_200.txt")
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text("\n\n".join(dummy_emails))

print(f"Created {len(dummy_emails)} dummy emails in {out_path}")
print("Sample email:\n", dummy_emails[0])

Created 200 dummy emails in data\dummy_emails_200.txt
Sample email:
 Dear Customer, your payment of KES 11000.00 for account: PAYLEMAIYAN #F2 has been received from Lindsey Mcdonald 750****163 on 18/05/2025 09:44 AM. M-Pesa Ref: 9AS60PSOXL


* Logic engine

In [8]:
# --------------------------------------------------------
# Reads MPESA email notifications from dummy_emails.txt,
# Parses payment info, updates dummy_rent_tracker.xlsx,
# Avoids double-logging by checking ProcessedRefs sheet.

import pandas as pd
import re
from pathlib import Path
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
DATA_DIR = Path('data')
EMAIL_FILE = DATA_DIR / 'dummy_emails_200.txt'
SPREADSHEET_FILE = DATA_DIR / 'dummy_rent_tracker.xlsx'

# --- 1. Load Dummy Emails ---
with open(EMAIL_FILE, 'r') as f:
    email_texts = f.read().split('\n\n')

print(f"Loaded {len(email_texts)} emails.")

Loaded 200 emails.


In [9]:
email_texts[22]

'Dear Customer, your payment of KES 5000.00 for account: PAYLEMAIYAN #F2 has been received from David Brown 722****274 on 13/03/2025 08:01 PM. M-Pesa Ref: OTRGYGKLJ4'

In [10]:
# --- 2. Load Workbook and All Sheet Names ---
wb = load_workbook(SPREADSHEET_FILE)
sheet_names = wb.sheetnames

# --- 3. Load ProcessedRefs (deduplication) ---
try:
    processed_refs_df = pd.read_excel(SPREADSHEET_FILE, sheet_name='ProcessedRefs')
    processed_refs = set(str(ref).strip().upper() for ref in processed_refs_df['Ref'] if pd.notna(ref))
except Exception:
    processed_refs = set()
    print("ProcessedRefs sheet is empty or missing. Will create it.")

print(f"Found {len(processed_refs)} previously processed refs.")

Found 1250 previously processed refs.


In [11]:
# --- 4. Regex Parser Function ---
def extract_payment_info(email_body):
    pattern = (
        r'payment of KES ([\d,]+\.\d{2}) '
        r'for account: PAYLEMAIYAN\s*#?\s*([A-Za-z]\d{1,2})'
        r' has been received from (.+?) '
        r'(.{1,13}) '
        r'on (\d{2}/\d{2}/\d{4} \d{1,2}:\d{2} [APM]{2})\. '
        r'M-Pesa Ref: ([\w\d]+)'
    )
    match = re.search(pattern, email_body, flags=re.IGNORECASE)
    if match:
        return {
            'Amount': float(match.group(1).replace(',', '').strip()),
            'AccountCode': match.group(2).strip().upper(),
            'Payer': match.group(3).strip(),
            'Phone': match.group(4).strip(),
            'Date': match.group(5).strip(),
            'Ref': match.group(6).strip().upper(),
        }
    return None

In [12]:
# --- 5. Process Emails, Update or Create Sheets ---


updates_log = []
new_refs = []
updates_per_sheet = {}

# We'll use openpyxl to add new sheets if needed
wb = load_workbook(SPREADSHEET_FILE)
writer = pd.ExcelWriter(SPREADSHEET_FILE, engine='openpyxl', mode='a', if_sheet_exists='overlay')

# Loading a Master payments file
try:
    payment_history_df = pd.read_excel(SPREADSHEET_FILE, sheet_name='PaymentHistory')
except Exception:
    payment_history_df = pd.DataFrame(columns=[
        'Date', 'Amount', 'Ref', 'Payer', 'Phone', 'Payment Mode', 'AccountCode', 'TenantSheet'
    ])


for email in email_texts:
    payment_data = extract_payment_info(email)
    if not payment_data:
        updates_log.append("Skipped email: Could not parse payment info.")
        continue

    ref = payment_data['Ref'].upper().strip()
    if ref in processed_refs:
        updates_log.append(f"Duplicate ignored (Ref {ref})")
        continue

    account_code = payment_data['AccountCode']
    payer_name = payment_data['Payer'].replace(" ", "_")[:15]
    # Try to match an existing tenant sheet
    target_sheet = None
    for s in sheet_names:
        # Take just the code part from the sheet name
        sheet_token = s.split()[0].replace('-', '').upper().strip()
        if account_code == sheet_token and 'PROCESSEDREFS' not in s.upper() and 'PAYMENTHISTORY' not in s.upper():
            target_sheet = s
            break

    # --- 7. If no sheet found, CREATE it ---
    if target_sheet is None:
        target_sheet = f"{account_code} - {payer_name if payer_name else 'AutoAdded'}"
        print(f"Creating new sheet: {target_sheet} for new tenant {account_code}")
        new_tenant_df = pd.DataFrame(columns=[
            'Date', 'Amount', 'Ref', 'Payer', 'PhoneOrID', 'Payment Mode'
        ])
        new_tenant_df.to_excel(writer, sheet_name=target_sheet, index=False)
        updates_log.append(f"Created new sheet: {target_sheet}")
        sheet_names.append(target_sheet)  # So we don't create it twice

    # --- 8. Append payment to tenant sheet ---
    try:
        df = pd.read_excel(SPREADSHEET_FILE, sheet_name=target_sheet)
    except Exception:
        df = pd.DataFrame(columns=['Date', 'Amount', 'Ref', 'Payer', 'PhoneOrID', 'Payment Mode'])

    new_row = pd.DataFrame({
        'Date': [payment_data['Date']],
        'Amount': [payment_data['Amount']],
        'Ref': [payment_data['Ref']],
        'Payer': [payment_data['Payer']],
        'PhoneOrID': [payment_data['PhoneOrID']],
        'Payment Mode': ['MPESA Payment'],
    })
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_excel(writer, sheet_name=target_sheet, index=False)
    updates_log.append(f"Logged payment for {account_code} - Ref {ref}")
    new_refs.append(ref)
    updates_per_sheet.setdefault(target_sheet, 0)
    updates_per_sheet[target_sheet] += 1

     # --- 9. Add to PaymentHistory sheet ---
    new_hist_row = new_row.copy()
    new_hist_row['AccountCode'] = account_code
    new_hist_row['TenantSheet'] = target_sheet
    payment_history_df = pd.concat([payment_history_df, new_hist_row], ignore_index=True)

KeyError: 'PhoneOrID'

In [ ]:
# --- 10. Save PaymentHistory sheet
payment_history_df.to_excel(writer, sheet_name='PaymentHistory', index=False)

# --- 11. Update ProcessedRefs sheet
try:
    refs_df = pd.read_excel(SPREADSHEET_FILE, sheet_name='ProcessedRefs')
except Exception:
    refs_df = pd.DataFrame({'Ref': []})
if new_refs:
    new_refs_df = pd.DataFrame({'Ref': new_refs})
    updated_refs = pd.concat([refs_df, new_refs_df], ignore_index=True)
    updated_refs.to_excel(writer, sheet_name='ProcessedRefs', index=False)
    updates_log.append(f"ProcessedRefs updated with {len(new_refs)} new refs.")

writer.close()

print("\n--- Processing Summary ---")
for log in updates_log:
    print(log)
print("\nUpdates per tenant sheet:")
for k, v in updates_per_sheet.items():
    print(f"{k}: {v} payments appended")


--- Processing Summary ---
Logged payment for C3 - Ref EN2S5W1K15
Logged payment for E3 - Ref TDPBK94WB3
Logged payment for C3 - Ref 9R1ZXBXIEJ
Logged payment for E1 - Ref 1A2SAZ6ZXY
Logged payment for E4 - Ref MALRGW9Q2J
Logged payment for E6 - Ref OUHOP1L40S
Logged payment for C5 - Ref 6UC0B4NYFU
Logged payment for D2 - Ref WNL2SRV9N6
Logged payment for B1 - Ref 5TI40XT1D4
Logged payment for E1 - Ref 2C3KQZ7RV0
Logged payment for C6 - Ref Q58O0PQH3Y
Logged payment for C1 - Ref V4NRV9PA11
Logged payment for B5 - Ref JBWDP1V1PC
Logged payment for B3 - Ref AT43F9BAA6
Logged payment for A5 - Ref EHAUX3TWV0
Logged payment for A3 - Ref FSYAFA45RE
Logged payment for C2 - Ref X9NNQ4GARU
Logged payment for F3 - Ref LDXV7I787L
Logged payment for F4 - Ref VDYRMLQQZF
Logged payment for A5 - Ref JJAUPU41ZN
Logged payment for B1 - Ref L6IAV493B6
Logged payment for B5 - Ref J7BQ1QBTCE
Logged payment for D2 - Ref P3H9WOYU37
Logged payment for G3 - Ref 09W6DXTO2R
Logged payment for B5 - Ref WL1BPVYS

## PROTOTYPE

* Intergrating the proof of concept to the Google Platform
* First is to create a dummy account on gmail and populate it with dummy emails as above.

### DUMMY EMAIL GENERATION

In [3]:
# Loading dependencies for sending email notifications
import base64, random, string, time, datetime
from faker import Faker
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText


In [7]:
# ---------- SEND 200 TEST EMAILS INTO SANDBOX GMAIL ----------

fake = Faker()
SCOPES = SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',     # to inject test mail
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.modify' # to call getProfile
]
flow   = InstalledAppFlow.from_client_secrets_file('bot_secret.json', SCOPES)
creds  = flow.run_local_server(port=0)
gmail  = build('gmail', 'v1', credentials=creds)
user_email = gmail.users().getProfile(userId='me').execute()['emailAddress']

accounts = [f"{l}{n}" for l in "ABCDEFG" for n in range(1,7)]
def rand_ref(): return ''.join(random.choices(string.ascii_uppercase+string.digits, k=10))

def make_msg(text):
    m = MIMEText(text)
    m['From'] = 'NCB <ncbcustomer@ncbgroup.com>'
    m['To']   = user_email
    m['Subject'] = 'NCBA TRANSACTIONS STATUS UPDATE'
    return {'raw': base64.urlsafe_b64encode(m.as_bytes()).decode()}

for _ in range(40):
    code  = random.choice(accounts)
    code_fragment = f"#{code}" if random.random()>.4 else code   # hash optional
    amt   = f"{random.randint(5,20)*1000:,}.00"
    name  = fake.name().upper()
    phone = f"0{random.randint(100,999)}***{random.randint(100,999)}"
    dt    = fake.date_time_this_year().strftime('%d/%m/%Y %I:%M %p')
    ref   = rand_ref()
    body  = (f"Your M-Pesa payment of KES {amt} for account: PAYLEMAIYAN {code_fragment} "
             f"has been received from {name} {phone} on {dt}. M-Pesa Ref: {ref}. NCBA, Go for it.")
    gmail.users().messages().send(userId='me', body=make_msg(body)).execute()
    time.sleep(0.3) # Adjusted sleep time to avoid rate limits

print("✅ 40 dummy messages delivered to", user_email)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899105285450-50tdk35cnnrrich3nlr0d80kdp2qeovr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64255%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=TYzmv311ajdCSUiS6AZT5qGNJ4rnHR&access_type=offline
✅ 200 dummy messages delivered to dmmccntdev@gmail.com


### GOOGLE SHEET GENERATION

In [2]:
# ---------- LOAD GSPREAD LIBRARIES FOR GOOGLE SHEETS ----------
import pandas as pd, gspread, openpyxl
from google.oauth2.service_account import Credentials

In [5]:
# ---------- ONE‑TIME MIGRATION EXCEL → GOOGLE SHEETS ----------


SRC_EXCEL = 'data/2025 RENT TRACKING - Lemaiyan Heights.xlsx'  # original data file
DEST_SHEET = 'RENT TRACKING-Lemaiyan Heights' # New file in google sheets

creds = Credentials.from_service_account_file('bot_service.json',
    scopes=['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'])
gc = gspread.authorize(creds)
sh = gc.open(DEST_SHEET)

wb = openpyxl.load_workbook(SRC_EXCEL, data_only=True)
for ws in wb.worksheets:
    title = ws.title[:99]  # Sheets title limit
    if title in [s.title for s in sh.worksheets()]:
        sheet = sh.worksheet(title)
    else:
        sheet = sh.add_worksheet(title, rows=2000, cols=10)

    data = [[str(cell) if cell is not None else '' for cell in row] for row in ws.iter_rows(values_only=True)]
    
#   Update the sheets to populate
    sheet.update(values=data, range_name='A1', value_input_option='USER_ENTERED')
    time.sleep(2)  # Wait 2 seconds per write

    # freeze first 7 rows and bold headers
    sheet.format('1:7', {'textFormat': {'bold': True}})
    sheet.freeze(rows=1)

print("Bootstrap complete – Google Sheet mirrors the Excel file.")


Bootstrap complete – Google Sheet mirrors the Excel file.


### BOT-SERVICE

In [9]:
# ================================
#  RENT RPA — PROTOTYPE
#  Gmail -> Google Sheets
# ================================

import re, base64, time
import pandas as pd
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from IPython.display import display

# --- Google APIs ---
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.service_account import Credentials
import gspread
from gspread.utils import rowcol_to_a1
from dateutil.relativedelta import relativedelta

# ---------------- CONFIG ----------------
CLIENT_SECRET = 'bot_secret.json'        # Gmail OAuth Desktop credentials
SERVICE_KEY   = 'bot_service.json'      # Sheets service account (shared on the target Sheet)
SHEET_NAME    = 'RENT TRACKING-Lemaiyan Heights'  # exact Google Sheet NAME
GMAIL_QUERY   = 'subject:"NCBA TRANSACTIONS STATUS UPDATE" newer_than:365d'  # tweak as needed

# This prototype uses a unified event schema for consistency:
PAYMENT_COLS  = ['Date Paid','Amount Paid','REF Number','Payer','Phone','Payment Mode']
MAX_PHONE_LEN = 13
REF_LEN       = 10


# ----- AUTH -----
gmail_flow = InstalledAppFlow.from_client_secrets_file(
    CLIENT_SECRET,
    scopes=[
        'https://www.googleapis.com/auth/gmail.modify',  # read + mark read
        'https://www.googleapis.com/auth/gmail.readonly',
        'https://www.googleapis.com/auth/gmail.send'
    ]
)
gmail_creds = gmail_flow.run_local_server(port=0)
gmail = build('gmail', 'v1', credentials=gmail_creds)

sheets_creds = Credentials.from_service_account_file(
    SERVICE_KEY,
    scopes=['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'])
gc = gspread.authorize(sheets_creds)
sh = gc.open(SHEET_NAME)

# ----- PARSER (flexible account code; 10-char ref) -----
PATTERN = re.compile(
    rf'payment of KES ([\d,]+\.\d{{2}}) '
    rf'for account: PAYLEMAIYAN\s*#?\s*([A-Za-z]\d{{1,2}})'
    rf' has been received from (.+?) '
    rf'(.{{1,{MAX_PHONE_LEN}}}) '
    rf'on (\d{{2}}/\d{{2}}/\d{{4}} \d{{1,2}}:\d{{2}} [APM]{{2}})\. '
    rf'M-Pesa Ref: ([A-Z0-9]{{{REF_LEN}}})',
    flags=re.IGNORECASE
)

def parse_email(text: str):
    m = PATTERN.search(text or "")
    if not m:
        return None
    amt, code, payer, phone, dt, ref = m.groups()
    return {
        'Date Paid':   dt.strip(),                      # dd/mm/YYYY hh:mm AM/PM
        'Amount Paid': float(amt.replace(',', '')),
        'REF Number':  ref.upper(),
        'Payer':       payer.strip(),
        'Phone':       phone.strip(),
        'Payment Mode':'MPESA Payment',
        'AccountCode': code.upper(),                    # used for routing to the tenant sheet
    }

# ----- GMAIL: get message text (best-effort) -----
def get_message_text(service, msg_id):
    msg = service.users().messages().get(userId="me", id=msg_id, format="full").execute()
    payload = msg.get("payload", {})
    body_texts = []

    def walk(part):
        mime = part.get("mimeType", "")
        data = part.get("body", {}).get("data")
        parts = part.get("parts", [])
        if mime == "text/plain" and data:
            body_texts.append(base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore"))
        for p in parts:
            walk(p)

    walk(payload)
    if body_texts:
        return "\n".join(body_texts)
    return msg.get("snippet", "")

# Normalizer: lower, trim, collapse spaces, strip punctuation/currency/nbsp
_PUNCT = re.compile(r"[^\w\s/]+", re.UNICODE)
def _norm(s):
    if s is None: return ""
    s = str(s).replace("\xa0", " ")  # nbsp -> space
    s = s.strip().lower()
    s = _PUNCT.sub("", s)            # remove punctuation like ( ), :, KES, etc.
    s = re.sub(r"\s+", " ", s)
    return s

# Broad alias sets (normalized)
ALIASES = {
    "month": {
        "month","month/period","period","rent month","billing month"
    },
    "amount_due": {
        "amount due","rent due","due","amountdue","monthly rent","rent","amount due kes","rent (kes)"
    },
    "amount_paid": {
        "amount paid","paid","amt paid","paid (kes)","amountpaid"
    },
    "date_paid": {
        "date paid","paid date","payment date","datepaid"
    },
    "ref": {
        "ref number","ref","reference","ref no","reference no","mpesa ref","mpesa reference","receipt","receipt no"
    },
    "date_due": {
        "date due","due date","rent due date","datedue"
    },
    "prepay_arrears": {
        "prepayment/arrears","prepayment","arrears","balance","bal","prepayment arrears","carry forward","cf"
    },
    "penalties": {
        "penalties","penalty","late fee","late fees","fine","fines"
    },
}

REQUIRED_KEYS = ["month","amount_due","amount_paid","date_paid","ref","date_due","prepay_arrears","penalties"]

# --- Helper to score header row ---
def _score_header(row_norm):
    """How many required columns does this row satisfy?"""
    hits = 0
    for key in REQUIRED_KEYS:
        if any(a in row_norm for a in ALIASES[key]):
            hits += 1
    return hits

# --- Helper to map row tokens to column keys ---
def _header_map_from_row(row):
    """Return (colmap) by matching normalized row tokens against aliases."""
    row_norm = [_norm(c) for c in row]
    colmap = {}
    for key, aliases in ALIASES.items():
        for i, token in enumerate(row_norm):
            if token in aliases:
                colmap[key] = i
                break
    return colmap


# --- Helper to detect or create a header row ---
def _detect_or_create_header(ws):
    """
    Find a header row in the first 10 rows.
    If none reaches a threshold (>=4 matches), insert a standard header at row 1.
    Returns (header_row_idx_0based, header_list, colmap).
    """
    all_data = ws.get_all_values()
    max_rows = len(all_data) if all_data else 1
    probe_rows = min(max_rows, 10)
    last_col = ws.col_count or 12
    rn = f"A1:{rowcol_to_a1(probe_rows, last_col)}"
    values = ws.get_values(rn)  # rectangular cut

    best_idx, best_hits, best_map = None, -1, None
    for idx, row in enumerate(values):
        colmap = _header_map_from_row(row)
        hits = len(colmap)
        if hits > best_hits:
            best_idx, best_hits, best_map = idx, hits, colmap

    if best_hits >= 4:
        header = ws.row_values(best_idx+1)
        missing_keys = [k for k in REQUIRED_KEYS if k not in best_map]
        if missing_keys:
            standard_columns = {
                "month": "Month",
                "amount_due": "Amount Due",
                "amount_paid": "Amount paid",
                "date_paid": "Date paid",
                "ref": "REF Number",
                "date_due": "Date due",
                "prepay_arrears": "Prepayment/Arrears",
                "penalties": "Penalties"
            }
            for key in missing_keys:
                header.append(standard_columns[key])
            ws.update(values=[header], range_name=f"{best_idx+1}:{best_idx+1}", value_input_option="USER_ENTERED")
            best_map = _header_map_from_row(header)
        return best_idx, header, best_map
    # No good header found: create standard header on row 1
    header = ['Month','Amount Due','Amount paid','Date paid','REF Number','Date due','Prepayment/Arrears','Penalties']
    if max_rows == 0:
        ws.update(values=[header], range_name="1:1", value_input_option="USER_ENTERED")
    else:
        ws.insert_row(header, index=1, value_input_option="USER_ENTERED")
    return 0, header, _header_map_from_row(header)


# --- Helper to convert date string to month key ---
def _month_key_from_date_str(date_str):
    dt = datetime.strptime(date_str, '%d/%m/%Y %I:%M %p')
    return dt.strftime('%B-%Y'), dt   # e.g., January-2025

# --- Helper to find the month row in values ---
def _find_month_row(values, month_col_idx, month_key):
    for r in range(1, len(values)):  # skip header at 0
        cell = str(values[r][month_col_idx]).strip()
        if not cell:
            continue
        # accept "Jan-2025"/"JAN 2025"/"January 2025"
        if cell.lower().startswith(month_key.lower()[:3]) and month_key[-4:] in cell:
            return r
    return None

# --- Helper to convert row/col to letter(s) ---
def _col_letter(row, col):
    """Return column letter(s) for a given 1-based row/col using A1 conversion."""
    return re.sub(r'\d+', '', rowcol_to_a1(row, col))


# --- Main function to update tenant month row ---
def update_tenant_month_row(tenant_ws, payment):
    """
    Realtime version:
      - Writes ONLY: Amount paid, Date paid, REF Number
      - Sets once-per-row formulas for:
          Prepayment/Arrears = N(Amount paid) - N(Amount Due)
          Penalties          = IF(DATEVALUE(LEFT(DatePaid,10)) > DATEVALUE(DateDue)+2, 3000, 0)
    """

    # --- detect/insert header (uses your robust detector from the previous block) ---
    header_row0, header, colmap = _detect_or_create_header(tenant_ws)
    missing = [k for k in REQUIRED_KEYS if k not in colmap]
    if missing:
        raise ValueError(f"Sheet '{tenant_ws.title}' missing required columns after normalization: {missing}")

    # Reload values from header row downward
    all_vals = tenant_ws.get_all_values()
    vals = all_vals[header_row0:]
    base_row_1based = header_row0 + 1

    # --- find or create the month row ---
    month_key, pay_dt = _month_key_from_date_str(payment['Date Paid'])
    row_rel = _find_month_row(vals, colmap['month'], month_key)
    if row_rel is None:
        new_row = [''] * len(header)
        new_row[colmap['month']] = month_key
        new_row[colmap['amount_due']] = '0'
        new_row[colmap['amount_paid']] = '0'
        new_row[colmap['date_paid']] = ''
        new_row[colmap['ref']] = ''
        # Set Date due as the previous row's date due plus one month.
        # Try to get last row's Date due (skip header row)
        if len(vals) > 1 and vals[-1][colmap['date_due']]:
            try:
                last_date_due = datetime.strptime(vals[-1][colmap['date_due']], "%d/%m/%Y").replace(day=5)
                new_date_due = last_date_due + relativedelta(months=1)
            except Exception:
            # Fallback to payment date plus one month if parsing fails
                new_date_due = datetime.strptime(payment['Date Paid'], '%d/%m/%Y %I:%M %p') + relativedelta(months=1)
        else:
            new_date_due = datetime.strptime(payment['Date Paid'], '%d/%m/%Y %I:%M %p') + relativedelta(months=1)
            new_row[colmap['date_due']] = new_date_due.strftime("%d/%m/%Y")
            
        # prepay/arrears and penalties will be set as FORMULAS after append
        tenant_ws.append_row(new_row, value_input_option='USER_ENTERED')
        all_vals = tenant_ws.get_all_values()
        vals = all_vals[header_row0:]
        row_rel = len(vals) - 1

    row_abs_1based = base_row_1based + row_rel
    row = vals[row_rel]

    # --- helpers to coerce numbers/strings ---
    def _num(v):
        try:
            s = str(v).replace(',','').strip()
            return float(s) if s else 0.0
        except:
            return 0.0
    def _str(v):
        return '' if v is None else str(v)

    # current row values
    due0   = _num(row[colmap['amount_due']])
    paid0  = _num(row[colmap['amount_paid']])
    ref0   = _str(row[colmap['ref']])

    pay_amt = float(payment['Amount Paid'])

    # (if you previously tracked arrears carryover in this cell, you can ignore that here
    #  because the balance is now a live formula: Paid - Due)
    paid1 = paid0 + pay_amt

    # --- 1) write the three direct fields ---
    updates = {
        colmap['amount_paid']:  paid1,
        colmap['date_paid']:    payment['Date Paid'],
        colmap['ref']:          (payment['REF Number'] if not ref0 else f"{ref0}, {payment['REF Number']}")
    }

    # compact range write
    touched = sorted(updates.keys())
    c1 = touched[0] + 1
    c2 = touched[-1] + 1
    rng = f"{rowcol_to_a1(row_abs_1based, c1)}:{rowcol_to_a1(row_abs_1based, c2)}"
    payload = [''] * (c2 - c1 + 1)
    for cidx, val in updates.items():
        payload[(cidx + 1 - c1)] = val
    payload = [str(x) if x is not None else '' for x in payload]

    for attempt in range(5):
        try:
            tenant_ws.update(values=[payload], range_name=rng, value_input_option='USER_ENTERED')
            break
        except HttpError as e:
            if getattr(e, "resp", None) and e.resp.status == 429:
                time.sleep(5 * (attempt+1))
                continue
            raise

    # --- 2) ensure the formula cells are present (set once; they’ll recalc automatically) ---
    col_letters = {k: _col_letter(row_abs_1based, colmap[k] + 1) for k in colmap}
    # addresses for this row:
    amt_paid_addr = f"{col_letters['amount_paid']}{row_abs_1based}"
    amt_due_addr  = f"{col_letters['amount_due']}{row_abs_1based}"
    date_paid_addr= f"{col_letters['date_paid']}{row_abs_1based}"
    date_due_addr = f"{col_letters['date_due']}{row_abs_1based}"
    bal_addr      = f"{col_letters['prepay_arrears']}{row_abs_1based}"
    pen_addr      = f"{col_letters['penalties']}{row_abs_1based}"


    # Penalties formula: if DatePaid > DateDue + 2 days, penalty = 3000
    pen_formula = f"=IF(DATEVALUE(LEFT({date_paid_addr},10))>DATEVALUE({date_due_addr})+2, 3000, 0)"

    # Balance formula: if first data row, =N(amt_paid)-N(amt_due); else, =N(prev_bal)+N(amt_paid)-N(amt_due)
    if row_abs_1based == base_row_1based:
        bal_formula = f"=N({amt_paid_addr})-N({amt_due_addr})-N({pen_addr})"
    else:
        prev_bal_addr = f"{col_letters['prepay_arrears']}{row_abs_1based-1}"
        bal_formula = f"=N({prev_bal_addr})+N({amt_paid_addr})-N({amt_due_addr})-N({pen_addr})"
    

    # Only set if not already a formula (so we don't overwrite intentional manual values)
    current_bal = tenant_ws.acell(bal_addr).value or ""
    current_pen = tenant_ws.acell(pen_addr).value or ""
    needs_bal = not str(current_bal).startswith("=")
    needs_pen = not str(current_pen).startswith("=")

    # Set any missing formulas in a single batch
    body = []
    if needs_bal:
        body.append({'range': bal_addr, 'values': [[bal_formula]]})
    if needs_pen:
        body.append({'range': pen_addr, 'values': [[pen_formula]]})
    if body:
        tenant_ws.batch_update(body, value_input_option='USER_ENTERED')

    # Return info (no computed numbers now—Sheet will reflect in realtime)
    return {
        'sheet': tenant_ws.title,
        'month_row': row_abs_1based,
        'paid_before': paid0,
        'paid_after': paid1,
        'ref_added': payment['REF Number'],
        'formulas_set': {'balance': needs_bal, 'penalties': needs_pen},
        'balance_addr': bal_addr,    
        'penalties_addr': pen_addr       
    }



# ----- META SHEETS (ProcessedRefs, PaymentHistory) -----
def ensure_meta(ws_name, header):
    try:
        ws = sh.worksheet(ws_name)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(ws_name, rows=2000, cols=max(10, len(header)))
        ws.append_row(header)
    return ws

refs_ws = ensure_meta("ProcessedRefs", ["Ref"])
hist_ws = ensure_meta("PaymentHistory", PAYMENT_COLS + ['AccountCode','TenantSheet','Month'])

# Load processed refs into a set
ref_vals = refs_ws.get_all_values()
processed_refs = set((r[0] or '').upper() for r in ref_vals[1:]) if len(ref_vals) > 1 else set()

# ----- GMAIL FETCH + PARSE -----
print("🔎 Searching Gmail…")
result = gmail.users().messages().list(userId="me", q=GMAIL_QUERY, maxResults=200).execute()
msg_list = result.get("messages", [])
print(f"Found {len(msg_list)} candidate emails.")

parsed, errors = [], []
for m in msg_list:
    try:
        text = get_message_text(gmail, m["id"])
        pay = parse_email(text)
        if not pay:
            errors.append(f"Could not parse message id {m['id']}")
            continue
        if pay['REF Number'] in processed_refs:
            continue
        parsed.append((m["id"], pay))
    except Exception as e:
        errors.append(f"Error reading message {m['id']}: {e}")

print(f"✅ Parsed {len(parsed)} new payments.")

# ----- APPLY: to tenant sheets + PaymentHistory + ProcessedRefs -----
logs = []
tenant_tally = {}

# Cache worksheets to reduce calls
worksheets = {ws.title: ws for ws in sh.worksheets()}

def find_or_create_tenant_sheet(account_code: str):
    for title, ws in worksheets.items():
        t = title.upper()
        if t.startswith(account_code) and 'PROCESSEDREFS' not in t and 'PAYMENTHISTORY' not in t:
            return ws
    title = f"{account_code} - AutoAdded"
    ws = sh.add_worksheet(title, rows=1000, cols=12)
    ws.update(values=[['Month','Amount Due','Amount paid','Date paid','REF Number','Date due','Prepayment/Arrears','Penalties']],
              range_name='A1', value_input_option='USER_ENTERED')
    ws.format('1:1', {'textFormat': {'bold': True}})
    ws.freeze(rows=1)
    worksheets[title] = ws
    logs.append(f"➕ Created tenant sheet: {title}")
    return ws

for msg_id, p in parsed:
    tenant_ws = find_or_create_tenant_sheet(p['AccountCode'])
    info = update_tenant_month_row(tenant_ws, p)
    # Read the live, recalculated values
    logs.append(
    f"🧾 {info['sheet']} R{info['month_row']} | "
    f"Paid {info['paid_before']}→{info['paid_after']} | "
    f"Ref {info['ref_added']} | Bal/penalties will auto-update in sheet"
    )
    
    tenant_tally[info['sheet']] = tenant_tally.get(info['sheet'], 0) + 1

    # PaymentHistory
    dt = datetime.strptime(p['Date Paid'], '%d/%m/%Y %I:%M %p')
    mon = dt.strftime('%Y-%m')
    hist_ws.append_row(
        [p[k] for k in PAYMENT_COLS] + [p['AccountCode'], tenant_ws.title, mon],
        value_input_option='USER_ENTERED'
    )

    # ProcessedRefs
    refs_ws.append_row([p['REF Number']], value_input_option='RAW')
    processed_refs.add(p['REF Number'])

    # Mark Gmail read (optional)
    try:
        gmail.users().messages().modify(userId='me', id=msg_id, body={'removeLabelIds': ['UNREAD']}).execute()
    except HttpError:
        pass

    time.sleep(2)  # throttle writes

# ----- GROUPED MONTHLY SUMMARY (display) -----
hist_vals = hist_ws.get_all_values()
if len(hist_vals) > 1:
    df = pd.DataFrame(hist_vals[1:], columns=hist_vals[0])
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        df['Amount Paid'] = pd.to_numeric(df['Amount Paid'], errors='coerce').fillna(0.0)
        grouped = df.groupby('Month', dropna=False).agg(
            Payments=('REF Number','count'),
            TotalAmount=('Amount Paid','sum')
        ).reset_index().sort_values('Month')
        display(grouped)
else:
    print("No payment history yet.")

# ----- LOGS -----
print("\n------ BOT LOG ------")
for line in logs:
    print(line)
print("\nPayments per tenant sheet:")
for t, c in tenant_tally.items():
    print(f"  {t}: {c} payment(s)")
if errors:
    print("\nNon-fatal parse/read issues:")
    for e in errors:
        print("  -", e)
print("\n✅ Prototype run complete.")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899105285450-50tdk35cnnrrich3nlr0d80kdp2qeovr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64549%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=RLSPJlljDsFVNMjhzgOmSIa4rLFqNV&access_type=offline
🔎 Searching Gmail…
Found 200 candidate emails.
✅ Parsed 0 new payments.


,Month,Payments,TotalAmount
0,2025-01,51,682000
1,2025-02,42,559000
2,2025-03,71,853000
3,2025-04,68,845000
4,2025-05,45,545000
5,2025-06,54,712000
6,2025-07,56,690000
7,2025-08,28,318000



------ BOT LOG ------

Payments per tenant sheet:

✅ Prototype run complete.


## DEPLOYMENT CODE

* In this integration, we extend the proof‐of‐concept prototype into a Streamlit app. The app retains the core functionality: • OAuth-based Google authentication to securely access Gmail, Google Sheets, and Drive. • Parsing payment notification emails with regular expressions. • Updating the relevant tenant sheets and maintaining meta data like ProcessedRefs and PaymentHistory.

* Streamlit provides a user-friendly dashboard where users can trigger the payment bot and view real-time summaries and logs. This separation of the UI from the backend logic enables rapid deployment and easier scalability.


In [ ]:
import streamlit as st
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
import gspread
from google.oauth2.credentials import Credentials
import pandas as pd
import re
import os
import pickle

# --- CONFIGURATION ---
CLIENT_SECRETS_FILE = 'client_secret.json'  # Download this from Google Cloud (OAuth Client ID)
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

SPREADSHEET_NAME = '2025 RENT TRACKING - Lemaiyan Heights'

# --- SESSION STATE ---
if 'credentials' not in st.session_state:
    st.session_state.credentials = None

# --- AUTHENTICATION FUNCTION ---
def authenticate_user():
    flow = Flow.from_client_secrets_file(
        CLIENT_SECRETS_FILE,
        scopes=SCOPES,
        redirect_uri='http://localhost:8501/'
    )
    auth_url, _ = flow.authorization_url(prompt='consent')

    st.write(f"[Click here to authorize access]({auth_url})")

    code = st.text_input('Paste the full redirect URL after authentication here:')
    if code:
        # Extract the code from URL
        parsed_code = code.split('code=')[1].split('&')[0]
        flow.fetch_token(code=parsed_code)
        creds = flow.credentials
        st.session_state.credentials = creds
        st.success('Authentication successful!')

# --- EMAIL PARSING LOGIC ---
def fetch_payment_emails(creds):
    service = build('gmail', 'v1', credentials=creds)
    results = service.users().messages().list(
        userId='me',
        q='from:ncbacustomer@ncbagroup.com subject:"NCBA TRANSACTIONS STATUS UPDATE"',
        maxResults=10
    ).execute()

    messages = results.get('messages', [])
    email_data = []

    for msg in messages:
        msg_content = service.users().messages().get(userId='me', id=msg['id']).execute()
        snippet = msg_content['snippet']
        email_data.append(snippet)

    return email_data

# --- PAYMENT DATA EXTRACTION ---
def extract_payment_info(email_body):
    pattern = r'payment of KES ([\d,]+.\d{2}) for account: ([\w#]+) has been received from (.+?) (\d{3}\*\*\*\*\d{3}) on (\d{2}/\d{2}/\d{4} \d{1,2}:\d{2} [APM]{2})\. M-Pesa Ref: ([\w\d]+)'
    match = re.search(pattern, email_body)
    if match:
        amount = float(match.group(1).replace(',', ''))
        account_code = match.group(2).split('#')[-1]
        payer_name = match.group(3)
        phone = match.group(4)
        payment_date = match.group(5)
        mpesa_ref = match.group(6)
        return {
            'Amount': amount,
            'AccountCode': account_code,
            'Payer': payer_name,
            'Phone': phone,
            'Date': payment_date,
            'Ref': mpesa_ref
        }
    return None

# --- GOOGLE SHEETS UPDATE FUNCTION ---
def update_google_sheet(creds, payment_data):
    gc = gspread.authorize(creds)
    sh = gc.open(SPREADSHEET_NAME)

    # Check or create 'ProcessedRefs' sheet
    try:
        refs_ws = sh.worksheet('ProcessedRefs')
    except gspread.exceptions.WorksheetNotFound:
        refs_ws = sh.add_worksheet(title='ProcessedRefs', rows="1000", cols="1")
        refs_ws.append_row(['Ref'])

    processed_refs = refs_ws.col_values(1)
    if payment_data['Ref'] in processed_refs:
        st.warning(f"Ref {payment_data['Ref']} already processed. Skipping.")
        return

    # Find Tenant Sheet
    tenant_ws = None
    for ws in sh.worksheets():
        if payment_data['AccountCode'] in ws.title and 'ProcessedRefs' not in ws.title:
            tenant_ws = ws
            break

    if tenant_ws:
        new_row = [
            payment_data['Date'],
            payment_data['Amount'],
            payment_data['Ref'],
            payment_data['Payer'],
            payment_data['Phone'],
            'MPESA Payment'
        ]
        tenant_ws.append_row(new_row)
        refs_ws.append_row([payment_data['Ref']])
        st.success(f"Payment logged for {payment_data['AccountCode']} successfully.")
    else:
        st.error(f"No matching tenant sheet found for {payment_data['AccountCode']}")

# --- MAIN APP LOGIC ---
st.title("🏠 Lemaiyan Heights Rent Automation Bot")

if not st.session_state.credentials:
    st.header("🔑 Authenticate with Google")
    authenticate_user()
else:
    st.success("You're authenticated!")

    if st.button("🚀 Run Payment Bot"):
        st.info("Fetching latest payment emails...")
        emails = fetch_payment_emails(st.session_state.credentials)

        if not emails:
            st.warning("No new payment emails found.")
        else:
            for email_body in emails:
                payment_info = extract_payment_info(email_body)
                if payment_info:
                    update_google_sheet(st.session_state.credentials, payment_info)
                else:
                    st.warning("Could not parse email. Possible format mismatch.")
